# Practice Querying the Snowexsql Database

(12 minutes)

Learning Objectives:  
- First taste of the database!
- Code snippets to extract and prep data.
- Generate ideas for project pitches.

In [ ]:
# standard imports
import numpy as np
import matplotlib.pyplot as plt 
import datetime

#database imports
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas

In [ ]:
# load the database
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)

print('snowexsql database successfully loaded!')

## Snow Pit data are contained in the following data tables:  

_PointData_  = pit ruler depths, SWE.  
_LayerData_  = density, temperature, stratigraphy, etc.  
_SiteData_   = siteID, airTemp, vegetation, visit time, weather, etc.  

### Example 1: Let's find all the pits that overlap with an airborne sensor of interest!

First, it would be helpful to know, which of the airborne sensors are part of the database, right?

In [ ]:
# Query the session using .surveyors() to generate a list
qry = session.query(ImageData.surveyors)

# Locate all that are distinct
airborne_sensors_list = session.query(ImageData.surveyors).distinct().all()

print('list of airborne sensors by "surveyor" name: \n', airborne_sensors_list)

#### 1a). Unsure of the flight date, but know which sensor you'd like to overlap with, here's how:

In [ ]:
# airborne sensor from list above
sensor = 'UAVSAR team, JPL'

# Form on the Images table that returns Raster collection dates
qry = session.query(ImageData.date)

# Filter for UAVSAR data
qry = qry.filter(ImageData.surveyors == sensor)

# Grab the unique dates
qry = qry.distinct()

# Execute the query 
dates = qry.all() 

# Clean up the dates 
dates = [d[0] for d in dates] 
dlist = [str(d) for d in dates]
dlist = ", ".join(dlist)
print('%s flight dates are: %s' %(sensor, dlist))

# Find all the snow pits done on these days
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(dates))

# return a geopandas df
df = query_to_geopandas(qry, engine)
df.head()

# Close your session to avoid hanging transactions
session.close()

#### 1b). Want to select an exact flight date match? Here's how:

In [ ]:
# Pick a day from the list of dates
dt = dates[0] 

# Find all the snow pits done on these days 
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date == dt)

# return a geopandas df
df_exact = query_to_geopandas(qry, engine)

print('%s pits overlap with %s on %s' %(len(df_exact), sensor, dt))
df_exact.head()

#### 1c). Want to select a range of dates near the flight date? Here's how:

In [ ]:
# Form a date range to query on either side of our chosen day 
date_range = [dt + i * datetime.timedelta(days=1) for i in [-1, 0, 1]]

# Find all the snow pits done on these days and return a geopandas df
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(date_range))

# return a geopandas df
df_range = query_to_geopandas(qry, engine)

# clean up dates (for print statement only)
dlist = [str(d) for d in date_range]
dlist = ", ".join(dlist)

print('%s pits overlap with %s on %s' %(len(df_range), sensor, dlist))
df_range

#### 1d). Have a known date in mind that you wish to select data for, here's how:

In [ ]:
# Find all the data that was collected on 2-12-2020
dt = datetime.date(2020, 2, 12)

# Grab all Point data instruments from our date
point_instruments = session.query(PointData.instrument).filter(PointData.date == dt).distinct().all()
print('Point Data on %s are: \n%s', (str(dt), point_instruments))

# Grab all Layer data instruments from our date
layer_instruments = session.query(LayerData.instrument, LayerData.type).filter(LayerData.date == dt).distinct().all()
print('\nLayer Data are: \n', layer_instruments)

# Grab all Image data instruments from our date
image_instruments = session.query(ImageData.instrument).filter(ImageData.date == dt).distinct().all()
print('\nImage Data are: \n', image_instruments)

qry = session.query(PointData.geom, PointData.instrument, PointData.site_id, PointData.date)
qry = qry.filter(PointData.date == dt) 
df = query_to_geopandas(qry, engine)
df

qry = session.query(LayerData.geom, LayerData.instrument, LayerData.site_id, LayerData.date)
qry = qry.filter(LayerData.date == dt) 
df = query_to_geopandas(qry, engine)
df

qry = session.query(ImageData.instrument, ImageData.site_id, ImageData.date)
qry = qry.filter(ImageData.date == dt) 
qry = qry.distinct()


In [ ]:

str(dt)

### Nice work, almost done here!

## Classify pit data based on the depth and vegetation matrix
### Example 2: 

#### 2a).Distinguish pits by vegetation coverage: 
- treeless (0% tree cover)
- sparse (1-30% tree cover)
- dense (31-100% tree cover)

*vegetation classes assigned based on optical imagery: tree density map, Nov. 2010 WorldView-2 Imagery

In [ ]:
def parse_veg_class(site_id):
    
        '''
    This function parses snow pit data into three vegetation classes:
        - 1). Treeless, 2). Sparce, and 3). Dense
        
    It uses a python dictionary where:
        (k) keys: are the vegetation classes
        (v) values: are the first digit in the pitID assignment

    
    '''
    
    # Classifying by vegetation coverage 
    veg_class = {'treeless':[1, 2, 3], 'sparse':[4, 5, 6], 'dense':[7, 8, 9]}
   
    vclass = None 
    
    class_id = site_id[0]
    
    if class_id.isnumeric():
        class_id = int(class_id)

        for k,v in veg_class.items():

            if class_id in v: #if the first digit in the site_id is 'v' assign it to the corresponding 'k'
                vclass = k 
                
    return vclass 

#### 2b). Distinguish pits by snow depth classes: 
- shallow (<90cm)
- medium (90-122cm)
- deep (>122cm)

*depth classes assigned based on 2017 snow depth lidar data

In [ ]:
def parse_depth_class(site_id):
    
            '''
    This function parses snow pit data into three depth classes:
        - 1). Shallow, 2). Medium, and 3). Deep
        
    It uses a python dictionary where:
        (k) keys: are the depth classes
        (v) values: are the first digit in the pitID assignment
      
  
    '''
        
    # Classifying by expected depth 
    depth_class = {'shallow':[1, 4, 7], 'medium':[2, 5, 8], 'deep':[3, 6, 9]} 
   
    dclass = None 
    
    class_id = site_id[0]
    
    if class_id.isnumeric(): #for the outlier TS site
        class_id = int(class_id) #cast as integer

        for k,v in depth_class.items(): #for the key, value pairs in the dict listed above:

            if class_id in v: #if the first digit in the site_id is 'v' assign it to the corresponding 'k'
                dclass = k 

    return dclass 

In [ ]:
# load the database
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)

result = session.query(LayerData.type).distinct().all()

qry = session.query(LayerData).filter(LayerData.type=='density')

# Form our dataframe from the query 
df = query_to_geopandas(qry, engine)
df['value'] = df['value'].astype(float) #cast the value as a float (they are strings)
 
# parse snow pit data by the veg/depth matrix
df['veg_class'] = [parse_veg_class(i) for i in df['site_id']] #run the parse_veg function for every site_id
df['depth_class'] = [parse_depth_class(i) for i in df['site_id']] #run the parse_depth funciton for every site_id

# # Show off our df 
# df.plot()

df.columns
col_list = ['site_name', 'date', 'id', 'instrument', 'type', 'units', 'surveyors', 'latitude',
       'longitude', 'geom','depth', 'site_id', 'value', 'veg_class', 'depth_class']
df = df[col_list]
df

In [ ]:
# These are all of the unique site ids 
len(df['site_id'].unique())

In [ ]:
#make df with only 153 rows using site_id.unique
gb = df.groupby(['site_id', 'veg_class'])
#gb = df.groupby('site_id')
gb['site_name'].count().groupby("veg_class").count()

In [ ]:
# d = df[['veg_class', 'depth_class']].groupby(df['site_id']).groups
# gb = df.groupby(['veg_class', 'site_id']).count()
# gb = df.groupby(['site_id']).count()
# df['veg_class'].groupby('veg_class').count()

# print(gb)

In [ ]:
# df['site_id'].unique().groupby('veg_class').count()

In [ ]:
df[['veg_class', 'site_id']].groupby('veg_class').count() #table I like!

In [ ]:
# Great for debugging especially when trying different queries
# session.rollback()